In [24]:
from langchain.tools import StructuredTool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
import pandas as pd
from openai import OpenAI
import os


In [25]:
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
import pandas as pd

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage

In [26]:

# Load and save functions
def load_inventory(filepath='inventory.csv'):
    return pd.read_csv(filepath)

def save_inventory(df, filepath='inventory.csv'):
    df.to_csv(filepath, index=False)


In [27]:

# Adjust quantity tool
class AdjustQuantityInput(BaseModel):
    item: str = Field(description="The name of the item to adjust in the inventory. Convert to singular form and correct grammar")
    quantity: int = Field(description="The quantity to adjust (positive to add, negative to remove)")

# Set quantity tool
class SetQuantityInput(BaseModel):
    item: str = Field(description="The name of the item to set in the inventory")
    quantity: int = Field(description="The total quantity to set for the item")

class CheckQuantity(BaseModel):
    item: str = Field(description="The name of the item to check in the inventory")


In [28]:

@tool("adjust-tool",args_schema=AdjustQuantityInput, return_direct=True)
def adjust_quantity(item: str, quantity: int) -> str:
    '''Adjust the quantity of an item in the inventory (add or remove).'''
    df = load_inventory()
    if item in df['item'].values:
        current_quantity = df.loc[df['item'] == item, 'quantity'].values[0]
        new_quantity = current_quantity + quantity
        if new_quantity < 0:
            df.loc[df['item'] == item, 'quantity'] = 0
            result = f"Warning: {item} cannot have a negative quantity. Set to 0 instead."
        else:
            df.loc[df['item'] == item, 'quantity'] = new_quantity
            result = f"Updated {item} by {quantity}. New quantity is {new_quantity}."
    else:
        df = pd.concat([df,pd.DataFrame([{'item': item, 'quantity': max(quantity, 0)}])], ignore_index=True)
        result = f"Added {item} to inventory with quantity {max(quantity, 0)}."
    save_inventory(df)
    return result

@tool("set-tool",args_schema=SetQuantityInput, return_direct=True)
def set_quantity(item: str, quantity: int) -> str:
    '''Set the total quantity of an item in the inventory.'''
    df = load_inventory()
    if item in df['item'].values:
        df.loc[df['item'] == item, 'quantity'] = quantity
        result = f"Set the quantity of {item} to {quantity}."
    else:
        df = pd.concat([df,pd.DataFrame([{'item': item, 'quantity': max(quantity, 0)}])], ignore_index=True)
        result = f"Added {item} to inventory with quantity {quantity}."
    save_inventory(df)
    return result

@tool("check-tool",args_schema=CheckQuantity, return_direct=True)
def check_quantity(item: str) -> str:
    '''Check and return the quantity of an item in the inventory.'''
    df = load_inventory()
    if item in df['item'].values:
        quantity = df.loc[df['item'] == item, 'quantity'] 
        result = f"The number of {item}s in the fridge is {quantity}."
    else:
        result = f"There are no {item}s in the fridge"
    save_inventory(df)
    return result

In [29]:
tools=[adjust_quantity, set_quantity, check_quantity]
tool_node = ToolNode(tools)

model = ChatOpenAI(model="gpt-4o", temperature=0).bind_tools(tools)


In [30]:

def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END

# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [31]:
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)


In [32]:
message = "How many apples are in the fridge?"
# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content=message)]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

'There are 16 apples in the fridge.'

In [5]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

llm = ChatOpenAI(model="gpt-4o", temperature=0)
tools=[adjust_quantity, set_quantity]
llm_with_tools = llm.bind_tools(tools)

# Create the agent
#agent = create_openai_functions_agent(llm, tools=[adjust_tool, set_tool], prompt=prompt)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)


# Create the executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Example input
response = agent_executor.invoke({"input": "I am putting 2 yoghurts, 5 apples and 2 eggs in the fridge"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `adjust-tool` with `{'item': 'yoghurt', 'quantity': 2}`


Updated yoghurt by 2. New quantity is 4.

Invoking: `adjust-tool` with `{'item': 'apple', 'quantity': 5}`


Updated apple by 5. New quantity is 13.

Invoking: `adjust-tool` with `{'item': 'egg', 'quantity': 2}`


Updated egg by 2. New quantity is 3.
I have updated the inventory:

- Yoghurt: Added 2 (Total: 4)
- Apples: Added 5 (Total: 13)
- Eggs: Added 2 (Total: 3)

> Finished chain.
{'input': 'I am putting 2 yoghurts, 5 apples and 2 eggs in the fridge', 'output': 'I have updated the inventory:\n\n- Yoghurt: Added 2 (Total: 4)\n- Apples: Added 5 (Total: 13)\n- Eggs: Added 2 (Total: 3)'}


In [ ]:

prompt = ChatPromptTemplate.from_template(
    "You are a fridge inventory manager. Your job is to update the inventory based on user inputs. "
    "Use the tools provided to make the necessary changes. Think about which tool is the most appropriate."
    "\n\nCurrent thoughts:\n{agent_scratchpad}\n\n"
    "User input: {input}\n"
    "Action: "
)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a fridge inventory manager. Your job is to update the inventory based on user inputs. "
            "Use the tools provided to make the necessary changes. Think about which tool is the most appropriate.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [6]:
response = agent_executor.invoke({"input": "I put 3 apples in the fridge and also took out 1 egg"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `adjust-tool` with `{'item': 'apple', 'quantity': 3}`


Updated apple by 3. New quantity is 16.

Invoking: `adjust-tool` with `{'item': 'egg', 'quantity': -1}`


Updated egg by -1. New quantity is 2.
The inventory has been updated:
- Added 3 apples. The new quantity is 16.
- Removed 1 egg. The new quantity is 2.

> Finished chain.
{'input': 'I put 3 apples in the fridge and also took out 1 egg', 'output': 'The inventory has been updated:\n- Added 3 apples. The new quantity is 16.\n- Removed 1 egg. The new quantity is 2.'}
